In [1]:
import openai

In [2]:
from openai import OpenAI

In [3]:
import os

In [5]:
client = OpenAI()

In [6]:
os.environ 

environ{'SHELL': '/bin/bash',
        'NUGET_XMLDOC_MODE': 'skip',
        'COLORTERM': 'truecolor',
        'CLOUDENV_ENVIRONMENT_ID': '4e7116b5-37a2-4817-b791-bdf15537d264',
        'NVM_INC': '/usr/local/share/nvm/versions/node/v20.14.0/include/node',
        'TERM_PROGRAM_VERSION': '1.90.2',
        'GITHUB_USER': 'Maxkaizo',
        'rvm_prefix': '/usr/local',
        'CODESPACE_NAME': 'refactored-pancake-5x7r79jg54wfv5gx',
        'HOSTNAME': 'codespaces-3e7f23',
        'JAVA_ROOT': '/home/codespace/java',
        'JAVA_HOME': '/usr/local/sdkman/candidates/java/current',
        'DOTNET_ROOT': '/usr/share/dotnet',
        'CODESPACES': 'true',
        'PYTHON_ROOT': '/home/codespace/.python',
        'GRADLE_HOME': '/usr/local/sdkman/candidates/gradle/current',
        'NVS_DIR': '/usr/local/nvs',
        'NVS_OS': 'linux',
        'DOTNET_SKIP_FIRST_TIME_EXPERIENCE': '1',
        'MY_RUBY_HOME': '/usr/local/rvm/rubies/ruby-3.2.4',
        'OPENAI_API_KEY': 'sk-proj-6yeZ2Ld3k3sf

In [8]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": "is it too late to join the course?"}]
)

In [9]:
response

ChatCompletion(id='chatcmpl-9dR0zbybmGQqAnvmCNqXAqYk2YuGS', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Whether or not it's too late to join a course depends on several factors, including the specific course's enrollment deadlines, the institution's policies, and the current point in the course schedule. Here are some steps you can take to find out:\n\n1. **Check the Enrollment Deadline**: Look for the course's enrollment or registration deadline on the institution's website or in the course catalog.\n\n2. **Contact the Instructor or Administration**: Reach out to the course instructor or the institution's administrative office. Explain your situation and inquire if late enrollment is possible.\n\n3. **Understand the Catch-Up Requirements**: If late enrollment is allowed, ask about what you need to do to catch up. This might include reviewing past lectures, completing missed assignments, or extra study sessions.\n\n4. **Consider 

In [10]:
! wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/minsearch.py

--2024-06-23 23:38:45--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-06-23 23:38:45 (37.2 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [11]:
import minsearch

In [15]:
!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/2d24c4f91fff80fe727eac7be73ccc06912f3137/01-intro/documents.json

--2024-06-23 23:40:24--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/2d24c4f91fff80fe727eac7be73ccc06912f3137/01-intro/documents.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 658332 (643K) [text/plain]
Saving to: ‘documents.json’

documents.json      100%[===================>] 642.90K  --.-KB/s    in 0.04s   

2024-06-23 23:40:24 (17.8 MB/s) - ‘documents.json’ saved [658332/658332]



In [16]:
import json

In [17]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [18]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [19]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [20]:
index = minsearch.Index(
    text_fields=["question","text","section"],
    keyword_fields=["course"]
)

In [21]:
q = 'the course has already startes, can I still enroll?'

In [22]:
index.fit(documents)

In [24]:
boost={'question': 3.0, 'section': 0.5}

result=index.search(
    query=q,
    filter_dict={'course': 'data-engineering-zoomcamp'},
    boost_dict=boost,
    num_results=5
)

In [25]:
result

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202